<a href="https://colab.research.google.com/github/DionnyZ/DIO--Treinamento-de-Redes-Neurais-com-Transfer-Learning/blob/main/Treinamento_de_Redes_Neurais_com_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNetV2
import numpy as np
import pandas as pd
import os
import zipfile

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data.zip to Data.zip


In [ ]:
with zipfile.ZipFile('Data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/content/Data/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    '/content/Data/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

Found 144 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.5938 - loss: 0.8063 - val_accuracy: 0.7500 - val_loss: 0.7339
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.4375 - loss: 1.7068 - val_accuracy: 1.0000 - val_loss: 0.1835
Epoch 3/10


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.8129 - loss: 0.4431
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 752ms/step - accuracy: 0.7500 - loss: 0.4672 - val_accuracy: 0.8750 - val_loss: 0.2306
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.8799 - loss: 0.2442 - val_accuracy: 1.0000 - val_loss: 0.1433
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8438 - loss: 0.2658
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.9615 - loss: 0.1317 - val_accuracy: 0.9375 - val_loss: 0.2012
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9375 - loss: 0.1784 - val_accuracy: 0.2500 - val_loss: 0.7883
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9737 - loss: 0.0869
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 879ms/step - accuracy: 1.0000 - loss: 0.0679 - val_accuracy: 0.9375 - val_loss: 0.2073


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/Data/TestDataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 20 images belonging to 2 classes.


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8000 - loss: 0.4454
Test accuracy: 0.800000011920929
Test loss: 0.445352703332901


In [ ]:
# Caminho da imagem a testar
img_path = '/content/Data/TestDataset/Jim Carrey/91.jpg'

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

prediction = model.predict(img_array)
print('Prediction:', prediction)

if prediction >= 0.5:
    print("A imagem é do Tom Hanks")
else:
    print("A imagem é do Jim Carrey")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Prediction: [[0.12104526]]
A imagem é do Jim Carrey
